<a href="https://colab.research.google.com/github/Chuck2Win/NH_project/blob/main/%5B%EB%8D%B0%EC%9D%B4%EC%BD%98%5D%5BBERT%5D%5B%EC%A4%91%EB%B3%B5%EC%A0%9C%EA%B1%B0%EC%97%86%EC%9D%B4%5D%5Boversampling%5D%5Bdistill%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip3 install transformers
! pip3 install imbalanced-learn
! pip3 install kobert-transformers
! pip3 install sentencepiece

     |████████████████████████████████| 1.8MB 13.0MB/s 
     |████████████████████████████████| 890kB 55.4MB/s 
     |████████████████████████████████| 2.9MB 58.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=479940b929c9d53c2c4a1f2f9fbd00a76e3c65b7e710def39ce0cc07047aeb43
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 12.5MB/s 


In [2]:
import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torchtext
from google.colab import drive
from transformers import AdamW,get_linear_schedule_with_warmup,get_constant_schedule_with_warmup
import time
import datetime
from torch.utils.data import TensorDataset, DataLoader
from pandas import DataFrame as df
import time,datetime
from sklearn.metrics import classification_report
from tqdm import tqdm
drive.mount('/content/gdrive')
os.chdir('./gdrive/My Drive/con')
import copy
from kobert_transformers import get_kobert_model,get_tokenizer
from transformers import get_cosine_schedule_with_warmup
tokenizer = get_tokenizer()
kobert = get_kobert_model()

Mounted at /content/gdrive


# 불러오기

In [3]:
train_data = pd.read_pickle('./sampled_train')
val_data = pd.read_pickle('./sampled_val')
test_data = pd.read_pickle('./test_data')

# TensorDataset으로 묶기

In [4]:
train_data = TensorDataset(torch.LongTensor(train_data['ids'].tolist()), torch.LongTensor(train_data['mask'].tolist()), torch.LongTensor(train_data['length'].tolist()), torch.LongTensor(train_data['longer'].tolist()), torch.LongTensor(train_data['shorter'].tolist()), torch.FloatTensor(train_data['logits'].tolist()), torch.LongTensor(train_data['info'].tolist()))
val_data = TensorDataset(torch.LongTensor(val_data['ids'].tolist()), torch.LongTensor(val_data['mask'].tolist()), torch.LongTensor(val_data['length'].tolist()), torch.LongTensor(val_data['longer'].tolist()), torch.LongTensor(val_data['shorter'].tolist()), torch.FloatTensor(val_data['logits'].tolist()), torch.LongTensor(val_data['info'].tolist()))
test_data = TensorDataset(torch.LongTensor(test_data['ids'].tolist()), torch.LongTensor(test_data['mask'].tolist()), torch.LongTensor(test_data['length'].tolist()), torch.LongTensor(test_data['longer'].tolist()), torch.LongTensor(test_data['shorter'].tolist()), torch.FloatTensor(test_data['logits'].tolist()), torch.LongTensor(test_data['info'].tolist()))

# 모델

In [5]:
class kobert_classifier(nn.Module):
    def __init__(self, kobert):
        super().__init__()
        self.bert = kobert
        self.classifier = nn.Linear(768+3,2)
    def forward(self,input_ids,attention_mask,length, longer, shorter):
        output = self.bert.forward(input_ids = input_ids, attention_mask = attention_mask)
        input = torch.cat([output.pooler_output, length.unsqueeze(-1),longer.unsqueeze(-1) ,shorter.unsqueeze(-1)],-1) # bs, 768
        predict = self.classifier.forward(input)
        return predict

In [6]:
class student_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(8002,128,padding_idx = 1)
        self.lstm = nn.LSTM(128,128,bidirectional=True, batch_first=True)
        self.linear = nn.Linear(256+3,2)
        self.dropout = nn.Dropout(0.1)
    def forward(self, ids, attention_mask,length,longer,shorter):
        out = self.embedding(ids)
        out = self.dropout(out)
        out,state = self.lstm(out) # out : (bs, seq_len, d_model*2) # hidden_state : ((2,bs,d_model))
        hidden = state[0].transpose(0,1).reshape(-1,2*128) # bs, 2*d_model
        input = torch.cat([hidden, length.unsqueeze(-1),longer.unsqueeze(-1) ,shorter.unsqueeze(-1)],-1) # bs, 768
        input = self.dropout(input)
        output = self.linear(input)
        return output

# teacher logit predict

In [7]:
# 배치 사이즈
batch_size = 256
train_dataloader = DataLoader(train_data, batch_size=batch_size,drop_last=False)
val_dataloader = DataLoader(val_data, batch_size=batch_size,drop_last=False)
test_dataloader = DataLoader(test_data, batch_size=batch_size,drop_last=False)

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = student_model().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.1)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-8, last_epoch=-1, verbose=False)    
a=1.
epochs=200

In [11]:
#os.mkdir('./distill_')
os.chdir('./distill_')

In [129]:
device='cuda:0'
min_value = float('inf')
min_epoch = None
min_model = None
count = 0

for epoch in tqdm(range(1, epochs+1),desc='epoch',mininterval = 300):
    # ========================================
    #               Training
    # ========================================
    total_loss = 0
    Predicted=[]
    Actual=[]
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, length, longer, shorter, logits,labels = batch
        outputs = model.forward(input_ids, attention_mask, length, longer, shorter)
        soft_loss = F.mse_loss(outputs, logits)
        hard_loss = F.cross_entropy(outputs, labels)
        loss = soft_loss * a + hard_loss * (1.-a)
        Predicted.extend(outputs.argmax(-1).tolist())
        Actual.extend(labels.tolist())
        total_loss+=loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    scheduler.step()
    avg_train_loss = total_loss/len(train_dataloader)
    if epoch%5 == 0:
        print(epoch)
        torch.save(model.state_dict(),'./epoch_%d_model'%(epoch))
        with torch.no_grad():
            model.eval()
            total_loss = 0
            Predicted=[]
            Actual=[]
            for batch in train_dataloader:
                batch = tuple(t.to(device) for t in batch)
                input_ids, attention_mask, length, longer, shorter, logits,labels = batch
                outputs = model.forward(input_ids, attention_mask, length, longer, shorter)
                soft_loss = F.mse_loss(outputs, logits)
                hard_loss = F.cross_entropy(outputs, labels)
                loss = soft_loss * a + hard_loss * (1-a)
                Predicted.extend(outputs.argmax(-1).tolist())
                Actual.extend(labels.tolist())
                total_loss+=loss
            acc = (np.array(Actual)==np.array(Predicted)).mean()
            train_acc.append(acc)
            
            curr_loss = total_loss / len(train_dataloader)     
            train_loss.append(curr_loss)
            print("")
            print("  Average train loss: {0:.5f}".format(curr_loss))
            print(classification_report(Actual,Predicted,digits=4))
            print("")
        
        with torch.no_grad():
            model.eval()
            total_loss = 0
            Predicted=[]
            Actual=[]
            for batch in val_dataloader:
                batch = tuple(t.to(device) for t in batch)
                input_ids, attention_mask, length, longer, shorter, logits,labels = batch
                outputs = model.forward(input_ids, attention_mask, length, longer, shorter)
                soft_loss = F.mse_loss(outputs, logits)
                hard_loss = F.cross_entropy(outputs, labels)
                loss = soft_loss * a + hard_loss * (1-a)
                Predicted.extend(outputs.argmax(-1).tolist())
                Actual.extend(labels.tolist())
                total_loss+=loss
            acc = (np.array(Actual)==np.array(Predicted)).mean()
            val_acc.append(acc)
            
            curr_loss = total_loss / len(val_dataloader)        
            val_loss.append(curr_loss)    
            print("")
            print("  Average val loss: {0:.5f}".format(curr_loss))
            print(classification_report(Actual,Predicted,digits=4))
            print("")
            if min_epoch is None:
                min_epoch = copy.copy(epoch)
                min_value = copy.copy(curr_loss)
                min_model = copy.deepcopy(model.state_dict())
            else:
                if min_value < curr_loss:
                    count+=1
                    if count == 4:
                        torch.save(min_model, './at_%d_model'%(min_epoch))
                        break
                else:
                    count = 0 
                    min_value = curr_loss
                    min_epoch = copy.copy(epoch)
                    min_model = copy.deepcopy(model.state_dict())    









epoch:   0%|          | 0/200 [00:00<?, ?it/s]

5

  Average train loss: 26.83105
              precision    recall  f1-score   support

           0     0.9959    0.1763    0.2995     68238
           1     0.5482    0.9993    0.7080     68238

    accuracy                         0.5878    136476
   macro avg     0.7720    0.5878    0.5037    136476
weighted avg     0.7720    0.5878    0.5037    136476



  Average val loss: 25.12236
              precision    recall  f1-score   support

           0     1.0000    0.1836    0.3102      2315
           1     0.5505    1.0000    0.7101      2315

    accuracy                         0.5918      4630
   macro avg     0.7753    0.5918    0.5102      4630
weighted avg     0.7753    0.5918    0.5102      4630


10

  Average train loss: 1.51172
              precision    recall  f1-score   support

           0     0.9863    0.9510    0.9683     68238
           1     0.9527    0.9868    0.9694     68238

    accuracy                         0.9689    136476
   macro avg     0.9695    0









epoch:  17%|█▋        | 34/200 [05:02<24:34,  8.88s/it]

35

  Average train loss: 23.40845
              precision    recall  f1-score   support

           0     0.9990    0.2792    0.4365     68238
           1     0.5811    0.9997    0.7350     68238

    accuracy                         0.6395    136476
   macro avg     0.7900    0.6395    0.5857    136476
weighted avg     0.7900    0.6395    0.5857    136476



  Average val loss: 21.96349
              precision    recall  f1-score   support

           0     1.0000    0.2842    0.4427      2315
           1     0.5828    1.0000    0.7364      2315

    accuracy                         0.6421      4630
   macro avg     0.7914    0.6421    0.5895      4630
weighted avg     0.7914    0.6421    0.5895      4630












epoch:  17%|█▋        | 34/200 [05:21<24:34,  8.88s/it]

...

135

  Average train loss: 1.95305
              precision    recall  f1-score   support

           0     0.9992    0.9337    0.9654     68238
           1     0.9378    0.9993    0.9676     68238

    accuracy                         0.9665    136476
   macro avg     0.9685    0.9665    0.9665    136476
weighted avg     0.9685    0.9665    0.9665    136476



  Average val loss: 2.33302
              precision    recall  f1-score   support

           0     0.9962    0.9136    0.9531      2315
           1     0.9202    0.9965    0.9569      2315

    accuracy                         0.9551      4630
   macro avg     0.9582    0.9551    0.9550      4630
weighted avg     0.9582    0.9551    0.9550      4630












epoch:  67%|██████▋   | 134/200 [20:21<09:52,  8.98s/it]

140

  Average train loss: 0.08248
              precision    recall  f1-score   support

           0     0.9991    0.9981    0.9986     68238
           1     0.9981    0.9991    0.9986     68238

    accuracy                         0.9986    136476
   macro avg     0.9986    0.9986    0.9986    136476
weighted avg     0.9986    0.9986    0.9986    136476



  Average val loss: 0.54678
              precision    recall  f1-score   support

           0     0.9866    0.9866    0.9866      2315
           1     0.9866    0.9866    0.9866      2315

    accuracy                         0.9866      4630
   macro avg     0.9866    0.9866    0.9866      4630
weighted avg     0.9866    0.9866    0.9866      4630


145

  Average train loss: 1.29218
              precision    recall  f1-score   support

           0     0.9993    0.9565    0.9775     68238
           1     0.9583    0.9993    0.9784     68238

    accuracy                         0.9779    136476
   macro avg     0.9788    









epoch:  84%|████████▍ | 168/200 [25:16<04:48,  9.01s/it]







epoch:  84%|████████▍ | 168/200 [25:31<04:48,  9.01s/it]

170

  Average train loss: 0.06564
              precision    recall  f1-score   support

           0     0.9993    0.9985    0.9989     68238
           1     0.9986    0.9993    0.9989     68238

    accuracy                         0.9989    136476
   macro avg     0.9989    0.9989    0.9989    136476
weighted avg     0.9989    0.9989    0.9989    136476



  Average val loss: 0.56665
              precision    recall  f1-score   support

           0     0.9858    0.9870    0.9864      2315
           1     0.9870    0.9857    0.9864      2315

    accuracy                         0.9864      4630
   macro avg     0.9864    0.9864    0.9864      4630
weighted avg     0.9864    0.9864    0.9864      4630


175

  Average train loss: 1.38073
              precision    recall  f1-score   support

           0     0.9993    0.9532    0.9757     68238
           1     0.9553    0.9994    0.9768     68238

    accuracy                         0.9763    136476
   macro avg     0.9773    